In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from BCI_Functions import *
import torch
import torch.nn as nn
import torch.utils.data as Data
dataset_1=np.load('1.npy')
print(dataset_1.shape)
removed_label=[2,3,4,5,6,7,8,9,10]
for ll in removed_label:
    id=dataset_1[:,-1]!=ll
    dataset_1 = dataset_1[id]
print(dataset_1.shape)
# data segmentation
n_class = int(11-len(removed_label))  # 0~9 classes ('10:rest' is not considered)
no_feature = 64  # the number of the features
segment_length = 8  # selected time window; 16=160*0.1
LR = 0.005  # learning rate
EPOCH = 401
data_seg=extract(dataset_1, n_classes=n_class, n_fea=no_feature, time_window=segment_length, moving=(segment_length/2))
print('After segmentation, the shape of the data:', data_seg.shape)
no_longfeature=no_feature*segment_length
data_seg_feature=data_seg[:,:no_longfeature]
data_seg_label=data_seg[:,no_longfeature:no_longfeature+1]
train_feature,test_feature,train_label,test_label=train_test_split(data_seg_feature,data_seg_label, shuffle=True)
train_feature_2d=train_feature.reshape([-1,no_feature])
test_feature_2d = test_feature.reshape([-1, no_feature])
print(data_seg_label)
from sklearn.preprocessing import MinMaxScaler
scaler3=MinMaxScaler().fit(train_feature)
train_fea_norm1=scaler3.transform(train_feature)
test_fea_norm1=scaler3.transform(test_feature)
print('After normalization, the shape of training feature:', train_fea_norm1.shape,
      '\nAfter normalization, the shape of test feature:', test_fea_norm1.shape)
train_fea_norm1=train_fea_norm1.reshape([-1,segment_length,no_feature])
test_fea_norm1=test_fea_norm1.reshape([-1,segment_length,no_feature])
print('After reshape, the shape of training feature:', train_fea_norm1.shape,
      '\nAfter reshape, the shape of test feature:', test_fea_norm1.shape)
BATCH_size=test_fea_norm1.shape[0]
print(train_label.shape)
print(BATCH_size)
train_fea_norm1=torch.tensor(train_fea_norm1).type('torch.FloatTensor')
train_label=torch.tensor(train_label.flatten())
train_data=Data.TensorDataset(train_fea_norm1,train_label)
train_loader=Data.DataLoader(dataset=train_data, batch_size=BATCH_size, shuffle=False)
test_fea_norm1=torch.tensor(test_fea_norm1).type('torch.FloatTensor')
test_label=torch.tensor(test_label.flatten())
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder=nn.Sequential(nn.Linear(no_feature*segment_length,64*4),)
        self.decoder=nn.Sequential(nn.Linear(64*4,no_feature*segment_length),nn.Sigmoid(),)
    def forward(self,x):
        encoded=self.encoder(x)
        decoded=self.decoder(encoded)
        return encoded,decoded
autoencoder=AutoEncoder()  
optimizer=torch.optim.Adam(autoencoder.parameters(),lr=LR)
loss_func=nn.MSELoss()

best_acc=[]

from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier(n_neighbors=3)

for epoch in range(EPOCH):
    for step, (train_x, train_y) in enumerate(train_loader):
        train_x=train_x.view(-1,no_feature*segment_length)
        train_encoded,train_decoded= autoencoder(train_x)
        loss=loss_func(train_decoded,train_x)
        optimizer.zero_grad
        loss.backward()
        optimizer.step() 
        if epoch % 50==0:
            knn.fit(train_encoded.data.numpy(),train_y.data.numpy())
            test_fea_norm1=test_fea_norm1.view(-1,no_feature*segment_length)
            test_encoded,test_decoded=autoencoder(test_fea_norm1)
            knn_acc=knn.score(test_encoded.data.numpy(),test_label.data.numpy())
            print('Epoch: ', epoch, '| STEP: ', step, '|Autoencoder train loss: %.4f' % loss.item(),'|KNN accuracy: %.4f' % knn_acc)
            best_acc.append(knn_acc)
            
print('BEST TEST ACC: {}'.format(max(best_acc)))



            


(259520, 65)
(19520, 65)
After segmentation, the shape of the data: (4880, 513)
[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]]
After normalization, the shape of training feature: (3660, 512) 
After normalization, the shape of test feature: (1220, 512)
After reshape, the shape of training feature: (3660, 8, 64) 
After reshape, the shape of test feature: (1220, 8, 64)
(3660, 1)
1220
Epoch:  0 | STEP:  0 |Autoencoder train loss: 0.0189 |KNN accuracy: 0.5082
Epoch:  0 | STEP:  1 |Autoencoder train loss: 0.0365 |KNN accuracy: 0.5156
Epoch:  0 | STEP:  2 |Autoencoder train loss: 0.0362 |KNN accuracy: 0.5033
Epoch:  50 | STEP:  0 |Autoencoder train loss: 0.2282 |KNN accuracy: 0.5262
Epoch:  50 | STEP:  1 |Autoencoder train loss: 0.2280 |KNN accuracy: 0.5631
Epoch:  50 | STEP:  2 |Autoencoder train loss: 0.2293 |KNN accuracy: 0.5574
Epoch:  100 | STEP:  0 |Autoencoder train loss: 0.2279 |KNN accuracy: 0.5426
Epoch:  100 | STEP:  1 |Autoencoder train loss: 0.2277 |KNN accuracy: 0.5770
Epoch:  100 | 